In [ ]:
using Revise
using FUSE
using Plots;
FUSE.logging(Logging.Info; actors=Logging.Info);

In [ ]:
# sty is the act equivalent for a study, it has common parameters like server and n_workers but also study dependent parameters like TGLF saturation rules
sty, act = FUSE.study_parameters(:TGLFdb);
sty

In [ ]:
using FUSE, SimulationParameters
sty, act = FUSE.study_parameters(Val(:TGLFdb))
:database_folder in propertynames(sty)    # should be true

In [ ]:
# Interacting with sty
sty.server = "saga"
sty.n_workers = 1

sty.database_folder = <Your database folder>

sty.save_folder = <Your save folder>
#study specific parameters
sty.sat_rules = missing  # missing (for custom models below) or [:sat1, :sat2, :sat3]
sty.custom_tglf_models = ["sat3_em_d3d_azf-1_withnegD"]
sty.file_save_mode = :overwrite
sty.release_workers_after_run = true # this is the default behavior and releases workers after running the study
sty

In [ ]:
# study is the actor equivalent of actors, here sty, act and outputs are kept of the workflow
study = FUSE.StudyTGLFdb(sty, act) # it is possible to pass in keyword arguments to sty

# instantiating the study will also setup the study i.e. FUSE.setup(study);

In [ ]:
study = FUSE.setup(study)

In [ ]:
# for now you will have to import FUSE everywhere in your distributed computing
using Distributed
@everywhere import FUSE

In [ ]:
study.act.ActorFluxMatcher.evolve_rotation = :flux_match
study.act.ActorFluxMatcher.rho_transport = 0.1:0.05:0.75
study.act.ActorFluxMatcher.max_iterations = 300
study.act.ActorFluxMatcher.algorithm = :simple_dfsane
study.act.ActorFluxMatcher.step_size = 1.0
study.act.ActorFluxMatcher

In [ ]:
FUSE.run(study);   # runs the study, this also writes the dataframes

# at the end of the run workers are released so you will have to setup again if you want to run more

In [ ]:
# data for this workflow is stored in a dictionary of dataframes study.dataframes_dict
study.dataframes_dict["sat3_em_d3d_azf-1_withnegD"]

In [ ]:
# analyze the study results 
FUSE.analyze(study);